In [2]:
import os
import sys
root_dir = os.path.abspath(os.path.join(os.path.dirname('../pruebillas.ipynb'), '..'))
os.chdir(root_dir)

sys.path.insert(0, os.path.join(root_dir, 'src'))

In [3]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, plot_heatmaps, resample_apply, barssince
import pandas as pd
import talib as ta
import pandas_ta as pandas_ta
import numpy as np
import matplotlib.pyplot as plt
from backtesting import Strategy
import plotly.express as px

from datetime import datetime
import MetaTrader5 as mt5
import pytz

from backbone.utils.general_purpose import diff_pips

In [4]:
def plot_stats(data, stats, strategy):
    equity_curve = stats._equity_curve
    aligned_data = data.reindex(equity_curve.index)
    bt = Backtest(aligned_data, strategy, cash=10_000_000, commission=0.002)
    print(stats)
    bt.plot(results=stats, resample=False)

In [53]:
def plot_full_equity_curve(data, stats_list, warmup_bars, lookback_bars, overlay_price=True):
    equity_curves = [x["_equity_curve"].iloc[warmup_bars:] for x in stats_list]

    combined = pd.Series(dtype=float)
    for curve in equity_curves:
        normalized_curve = curve["Equity"] / curve["Equity"].iloc[0]  # Normaliza la curva a su valor inicial
        if combined.empty:
            combined = normalized_curve
        else:
            # Alinea la nueva curva con la última de la serie combinada
            normalized_curve = normalized_curve * combined.iloc[-1]
            combined = pd.concat([combined, normalized_curve])

    fig = px.line(x=combined.index, y=combined)
    fig.update_traces(textposition="bottom right")
    fig.show()


In [6]:
# symbols_path = './backbone/data/backtest/symbols/USDCAD.csv'
# df = pd.read_csv(symbols_path)
# df = df[['Date','Open','High','Low','Close', 'Volume']]

print("MetaTrader5 package author: ", mt5.__author__)
print("MetaTrader5 package version: ", mt5.__version__)

# establish connection to MetaTrader 5 terminal
if not mt5.initialize():
    raise Exception("initialize() failed, error code =",mt5.last_error())

# set time zone to UTC
timezone = pytz.timezone("Etc/UTC")

# create 'datetime' objects in UTC time zone to avoid the implementation of a local time zone offset
utc_from = datetime(2024, 6, 1, tzinfo=timezone)
utc_to = datetime(2024, 8, 1, tzinfo=timezone)
rates = mt5.copy_rates_range('USDCAD', mt5.TIMEFRAME_M1
, utc_from, utc_to)

mt5.shutdown()

# create DataFrame out of the obtained data
df = pd.DataFrame(rates)

# convert time in seconds into the datetime format
df['time'] = pd.to_datetime(df['time'], unit='s')
                          
df = df.rename(columns={
  'time':'Date', 
  'open':'Open', 
  'high':'High', 
  'low':'Low', 
  'close':'Close', 
  'tick_volume':'Volume'
})

df

MetaTrader5 package author:  MetaQuotes Ltd.
MetaTrader5 package version:  5.0.4288


,Date,Open,High,Low,Close,Volume,spread,real_volume
0,2024-06-03 00:00:00,1.36310,1.36310,1.36310,1.36310,1,30,0
1,2024-06-03 00:03:00,1.36284,1.36284,1.36242,1.36256,4,85,0
2,2024-06-03 00:05:00,1.36256,1.36256,1.36256,1.36256,1,116,0
3,2024-06-03 00:06:00,1.36271,1.36276,1.36271,1.36273,3,67,0
4,2024-06-03 00:07:00,1.36273,1.36273,1.36256,1.36256,2,70,0
...,...,...,...,...,...,...,...,...
61152,2024-07-31 23:56:00,1.38066,1.38066,1.38056,1.38063,53,22,0
61153,2024-07-31 23:57:00,1.38063,1.38063,1.38053,1.38060,35,22,0
61154,2024-07-31 23:58:00,1.38060,1.38078,1.38060,1.38073,39,22,0
61155,2024-07-31 23:59:00,1.38072,1.38072,1.38061,1.38070,28,23,0


In [7]:
train_start = '2024-06-01'
train_end = '2024-07-01'

test_start = '2024-07-01'
test_end = '2024-07-15'

train_data = df[(df.Date > train_start) & (df.Date < train_end)]
test_data = df[(df.Date > test_start) & (df.Date < test_end)]

train_data.loc[:, 'Date'] = pd.to_datetime(train_data.Date)
test_data.loc[:, 'Date'] = pd.to_datetime(test_data.Date)

train_data = train_data.set_index('Date')
test_data = test_data.set_index('Date')

df = df.set_index('Date')


# ATR Strategy

In [8]:
def rolling_atr(atr, n):
    return atr.rolling(window=n).mean()

In [35]:

import random


class AtrStrategy(Strategy):
    position_size = 3500

    atr_window = 14
    atr_period = 5

    pip_size = 0.0001
    sl_pips = 10
    rr = 2

    random = None

    def init(self):
        self.atr = self.I(ta.ATR, self.data.High, self.data.Low, self.data.Close, timeperiod=self.atr_period)

    def next(self):
        close_prices = self.data.Close
        actual_close_price = close_prices[-1]
       
        if self.position:
            first_trade = self.trades[0]
            today = self.data.index[-1].tz_localize('UTC').tz_convert('UTC')
            time_in_position = (today - first_trade.entry_time.tz_localize('UTC').tz_convert('UTC'))
            time_in_position = time_in_position.total_seconds() // 60


            if time_in_position >= self.random:
                self.position.close()
                self.random = None
            pass
  
        else: 
            max_atr = max(self.atr[-self.atr_window:-1])

            if self.atr > max_atr and self.data.Open > self.data.Close:
                self.sell()
                self.random = random.randint(5, 20)

            elif self.atr > max_atr and self.data.Open < self.data.Close:

                self.buy()
                self.random = random.randint(5, 20)



In [38]:

bt_train = Backtest(
    train_data, 
    AtrStrategy, 
    cash=15_000, 
    # commission=0.0002,
    margin=1/30
)

stats = bt_train.optimize(
    # sl_pips=[6, 7, 8, 10],
    # rr=[0.5, 0.8, 1],
    atr_window = [8, 10, 12, 15, 20, 25, 30],
    atr_period = [10, 14, 16],
    maximize='Return [%]',
    max_tries=1000,
)

print(f'sl atr_window: {stats._strategy.atr_window}')
print(f'atr_period: {stats._strategy.atr_period}')

stats

  0%|          | 0/11 [00:00<?, ?it/s]

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_stats.py:122: RuntimeWarning: divide by zero encountered in scalar divide
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_stats.py:122: RuntimeWarning: divide by zero encountered in scalar divide
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501


sl atr_window: 8
atr_period: 10


Start                     2024-06-03 00:00:00
End                       2024-06-28 23:58:00
Duration                     25 days 23:58:00
Exposure Time [%]                    1.233538
Equity Final [$]                  16775.14169
Equity Peak [$]                   17144.41569
Return [%]                          11.834278
Buy & Hold Return [%]                0.336732
Return (Ann.) [%]                  379.369917
Volatility (Ann.) [%]              178.759346
Sharpe Ratio                         2.122238
Sortino Ratio                       29.028811
Calmar Ratio                        54.730533
Max. Drawdown [%]                   -6.931595
Avg. Drawdown [%]                   -2.075765
Max. Drawdown Duration       14 days 02:42:00
Avg. Drawdown Duration        1 days 13:56:00
# Trades                                   25
Win Rate [%]                             56.0
Best Trade [%]                       0.180431
Worst Trade [%]                     -0.070737
Avg. Trade [%]                    

In [39]:
bt_train.plot(filename='./atr.html', resample=False)


c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


GridPlot(id='p9356', ...)

In [41]:
bt_test = Backtest(
    test_data, 
    AtrStrategy, 
    cash=15_000, 
    # commission=0.0002,
    margin=1/30
)
 
test_stats = bt_test.run(
    sl_pips=stats._strategy.sl_pips,
    rr=stats._strategy.rr,
)



test_stats

Start                     2024-07-01 00:01:00
End                       2024-07-12 23:58:00
Duration                     11 days 23:57:00
Exposure Time [%]                    1.843588
Equity Final [$]                  16152.55769
Equity Peak [$]                   16202.25237
Return [%]                           7.683718
Buy & Hold Return [%]               -0.279403
Return (Ann.) [%]                  123.838229
Volatility (Ann.) [%]                95.19607
Sharpe Ratio                         1.300875
Sortino Ratio                        6.840182
Calmar Ratio                        16.039889
Max. Drawdown [%]                   -7.720641
Avg. Drawdown [%]                   -0.983859
Max. Drawdown Duration        8 days 00:39:00
Avg. Drawdown Duration        0 days 22:34:00
# Trades                                   17
Win Rate [%]                        58.823529
Best Trade [%]                       0.143169
Worst Trade [%]                     -0.092552
Avg. Trade [%]                    

In [11]:
bt_test.plot(filename='./ConsecutiveCandlesGridTest.html', resample=False)


c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


GridPlot(id='p2943', ...)

# WFO

In [44]:
def walk_forward(
        strategy,
        data_full,
        warmup_bars,
        lookback_bars=28*1440,
        validation_bars=7*1440,
        cash=15_000, 
        commission=0.002):

    stats_master = []

    for i in range(lookback_bars, len(data_full)-validation_bars, validation_bars):

        print(i)

        # To do anchored walk-forward, just set the first slice here to 0
        train_data = data_full.iloc[i-lookback_bars: i]

        bt_training = Backtest(train_data, strategy, cash=cash, commission=commission)
        stats_training = bt_training.optimize(
                atr_window = [8, 10, 12, 15, 20, 25, 30],
                atr_period = [10, 14, 16],
                maximize='Equity Final [$]')

        
        validation_data = data_full.iloc[i-warmup_bars:i+validation_bars]
        bt_validation = Backtest(validation_data, strategy, cash=cash, commission=commission)
        stats_validation = bt_validation.run(
                atr_window = stats_training._strategy.atr_window,
                atr_period = stats_training._strategy.atr_period)

        stats_master.append(stats_validation)

    return stats_master

In [45]:
import pickle


lookback_bars = 28*1440
validation_bars = 7*1440
warmup_bars = 14*60

if os.path.exists("stats.pickle"):
    with open("stats.pickle", "rb") as f:
        stats = pickle.load(f)
else:
    stats = walk_forward(AtrStrategy, df, warmup_bars = warmup_bars)
    # with open("stats.pickle", "wb") as f:
    #     pickle.dump(stats, f)

40320


  0%|          | 0/11 [00:00<?, ?it/s]

50400


  0%|          | 0/11 [00:00<?, ?it/s]

In [48]:
plot_stats(df, stats[1], AtrStrategy)

IndexError: list index out of range

In [54]:
plot_full_equity_curve(df, stats, warmup_bars = warmup_bars,
                       lookback_bars = lookback_bars, overlay_price = True)

In [26]:

import plotly.express as px

y = test_stats._equity_curve.Equity
x = test_stats._equity_curve.index


fig = px.line(x=x, y=y)
fig.update_traces(textposition="bottom right")
fig.show()

In [17]:
trades = test_stats._trades.groupby(by=['ExitTime']).agg({'PnL':['sum','count'], 'Duration':'max'})
trades.columns = trades.columns.droplevel(0)
trades = trades.reset_index().rename(columns={'count':'ammount_trades'})
trades = trades.rename(columns={'sum':'profit'})
trades = trades.rename(columns={'max':'minutes_in_trade'})
trades

,ExitTime,profit,ammount_trades,minutes_in_trade
0,2024-07-01 04:26:00,161.27713,1,0 days 00:06:00
1,2024-07-01 04:49:00,-206.35708,1,0 days 00:13:00
2,2024-07-01 08:52:00,-193.58018,1,0 days 02:21:00
3,2024-07-01 10:06:00,-197.64976,1,0 days 01:00:00
4,2024-07-01 10:35:00,150.17064,1,0 days 00:14:00
...,...,...,...,...
103,2024-07-12 15:26:00,134.73952,1,0 days 01:03:00
104,2024-07-12 17:23:00,-195.18906,1,0 days 00:11:00
105,2024-07-12 18:02:00,139.89600,1,0 days 00:30:00
106,2024-07-12 18:40:00,126.72530,1,0 days 00:30:00


In [18]:
trades.minutes_in_trade.describe()


count                          108
mean     0 days 01:09:38.333333333
std      0 days 01:43:26.515795336
min                0 days 00:01:00
25%                0 days 00:14:00
50%                0 days 00:30:30
75%                0 days 01:21:45
max                0 days 12:04:00
Name: minutes_in_trade, dtype: object